In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using Theano backend.


In [2]:
#IS2

In [3]:
#Read the data

GenomicFeatures_IS2 = pd.read_csv(open('data/IS2/GenomicFeatures_IS2.txt','r'),sep='\t')
SNVCalls_IS2    = pd.read_csv(open('data/IS2/SNVCalls_IS2.txt','r'),sep='\t') 

/usr/lib/python2.7/dist-packages/pandas/io/parsers.py:1070: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
print GenomicFeatures_IS2.shape
print SNVCalls_IS2.shape

(51108, 16)
(51108, 72)


In [5]:
np.sum(GenomicFeatures_IS2.isnull())

CHROM                 0
POS                   0
END                   0
RefAllele             0
NonRefAllele          0
BaseQual              0
TumourCoverage        0
NormalCoverage        0
MapQual               0
ReadPosition          0
Trinucleotide         0
HomopolymerRate       0
GCcontent             0
GenomicLocation    6460
Distance              0
StrandBias            0
dtype: int64

In [6]:
np.unique(GenomicFeatures_IS2['GenomicLocation'])

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_UTR5',
       'ncRNA_exonic', 'ncRNA_intronic', 'splicing', 'upstream',
       'upstream;downstream'], 
      dtype='|S19')

In [7]:
GenomicFeatures_IS2.head()

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,13142,13143,41,8,1670,56,70,0,21.0,TCA,2.567376,0.599010,intergenic,41571,0.375000
1,1,16976,16977,82,24,3377,126,142,0,37.5,GCC,2.468531,0.589109,ncRNA_exonic,37737,0.658730
2,1,17019,17020,89,38,3943,151,172,0,51.0,GCC,2.427586,0.589109,ncRNA_exonic,37694,0.509934
3,1,17384,17385,103,39,4198,164,170,0,44.0,CCC,2.463087,0.603960,ncRNA_intronic,37329,0.396341
4,1,18933,18934,36,16,1558,66,59,0,31.0,GCG,2.263158,0.589109,ncRNA_intronic,35780,0.363636


In [8]:
SNVCalls_IS2.head()

,CHROM,POS,END,X2357355,X2361924,X2361980,X2361993,X2361996,X2363577,X2363788,X2364661,X2364663,X2364695,X2366709,X2366711,X2367174,X2367176,X2367178,X2367180,X2367197,
0,1,13142,13143,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,1,16976,16977,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...
2,1,17019,17020,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...
3,1,17384,17385,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...
4,1,18933,18934,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...


In [9]:
np.unique(GenomicFeatures_IS2['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC', 'TCG',
       'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [11]:
IS2 = pd.merge(GenomicFeatures_IS2, SNVCalls_IS2,left_on=['CHROM', 'POS'], right_on=['CHROM','POS'])


In [12]:
IS2.head() #182+15

,CHROM,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,END_y,X2357355,X2361924,X2361980,
0,1,13142,13143,41,8,1670,56,70,0,21.0,TCA,2.567376,0.599010,intergenic,41571,0.375000,13143,0,0,0,...
1,1,16976,16977,82,24,3377,126,142,0,37.5,GCC,2.468531,0.589109,ncRNA_exonic,37737,0.658730,16977,0,0,0,...
2,1,17019,17020,89,38,3943,151,172,0,51.0,GCC,2.427586,0.589109,ncRNA_exonic,37694,0.509934,17020,0,0,0,...
3,1,17384,17385,103,39,4198,164,170,0,44.0,CCC,2.463087,0.603960,ncRNA_intronic,37329,0.396341,17385,0,0,0,...
4,1,18933,18934,36,16,1558,66,59,0,31.0,GCG,2.263158,0.589109,ncRNA_intronic,35780,0.363636,18934,0,0,0,...


In [13]:
IS2_X = IS2.iloc[:,3:16] #Exclude CHROM, POS
IS2_Y = IS2.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [14]:
print IS2_X.shape
print IS2_Y.shape

(42916, 13)
(42916, 69)


In [16]:
IS2_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,41,8,1670,56,70,0,21.0,TCA,2.567376,0.599010,intergenic,41571,0.37500
1,82,24,3377,126,142,0,37.5,GCC,2.468531,0.589109,ncRNA_exonic,37737,0.65873


In [18]:
IS2_Y[:2]

,X2357355,X2361924,X2361980,X2361993,X2361996,X2363577,X2363788,X2364661,X2364663,X2364695,X2366709,X2366711,X2367174,X2367176,X2367178,X2367180,X2367197,X2367207,X2367734,X2367761,
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [19]:
totalSample = IS2_Y.shape[0]
IS2_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print IS2_Y_Val.shape

42916
(42916, 1)


In [20]:
IS2_Y.shape

(42916, 69)

In [21]:
IS2_Y.values[0].shape

(69,)

In [22]:
for i in range(totalSample):
    IS2_Y_Val[i] = np.argmax(np.bincount(IS2_Y.values[i]))

In [23]:
IS2_Y_Val_1 = pd.DataFrame(IS2_Y_Val)
IS2_Y_Val_1.columns = ['Label']

In [24]:
IS2_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [25]:
IS2_dataframe = pd.concat([IS2_X, IS2_Y_Val_1],axis=1)

In [26]:
IS2_dataframe.shape

(42916, 14)

In [27]:
IS2_dataframe[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
0,41,8,1670,56,70,0,21.0,TCA,2.567376,0.599010,intergenic,41571,0.37500,0
1,82,24,3377,126,142,0,37.5,GCC,2.468531,0.589109,ncRNA_exonic,37737,0.65873,0


In [29]:
IS2_dataframe.to_csv('IS2.csv')

In [29]:
#Now we have variable importance ...lets do analysis

In [30]:
%load_ext rpy2.ipython

In [31]:
%%R
# Read in the data
IS2Frame = read.csv("IS2.csv")
str(IS2Frame)

'data.frame':	42916 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  41 82 89 103 36 53 80 44 47 44 ...
 $ NonRefAllele   : int  8 24 38 39 16 20 25 20 18 26 ...
 $ BaseQual       : int  1670 3377 3943 4198 1558 2231 3111 1885 1987 2129 ...
 $ TumourCoverage : int  56 126 151 164 66 85 118 75 74 77 ...
 $ NormalCoverage : int  70 142 172 170 59 87 136 85 63 81 ...
 $ MapQual        : num  0 0 0 0 0 0 17 0 0 0 ...
 $ ReadPosition   : num  21 37.5 51 44 31 61 49 57 49 52 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 25 18 18 10 19 12 22 27 15 15 ...
 $ HomopolymerRate: num  2.57 2.47 2.43 2.46 2.26 ...
 $ GCcontent      : num  0.599 0.589 0.589 0.604 0.589 ...
 $ GenomicLocation: Factor w/ 15 levels "","downstream",..: 5 7 7 8 8 8 8 8 5 5 ...
 $ Distance       : int  41571 37737 37694 37329 35780 34762 34578 32689 22247 21880 ...
 $ StrandBias     : num  0.375 0.659 0.51 0.396 0.364 ...
 $ Label          : num  0 0 0 0 0

In [32]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
#sum(is.na(IS1Frame$ReadPosition)) 
sum(is.na(IS2Frame)) 

[1] 0


In [33]:
#%%R
#CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
#CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

#print (sum(is.na(CPCG0235Frame$ReadPosition)) )
#print (sum(is.na(CPCG0235Frame$StrandBias)) )

In [33]:
%%R
sum(is.na(IS2Frame))  #No NA value now

[1] 0


In [34]:
%%R
write.csv(IS2Frame, file = "IS2_imputed.csv")  #Cleaned CSV without NA

In [35]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         10 minutes 14 seconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.58 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [36]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/IS2_imputed.csv"
IS2.hex          = h2o.importFile(path = csvPath, destination_frame = "IS2.hex") 
train                 = as.data.frame(IS2.hex)

str(train)

  |======================================================================| 100%
'data.frame':	42916 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  41 82 89 103 36 53 80 44 47 44 ...
 $ NonRefAllele   : int  8 24 38 39 16 20 25 20 18 26 ...
 $ BaseQual       : int  1670 3377 3943 4198 1558 2231 3111 1885 1987 2129 ...
 $ TumourCoverage : int  56 126 151 164 66 85 118 75 74 77 ...
 $ NormalCoverage : int  70 142 172 170 59 87 136 85 63 81 ...
 $ MapQual        : num  0 0 0 0 0 0 17 0 0 0 ...
 $ ReadPosition   : num  21 37.5 51 44 31 61 49 57 49 52 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 25 18 18 10 19 12 22 27 15 15 ...
 $ HomopolymerRate: num  2.57 2.47 2.43 2.46 2.26 ...
 $ GCcontent      : num  0.599 0.589 0.589 0.604 0.589 ...
 $ GenomicLocation: Factor w/ 14 levels "downstream","exonic",..: 4 6 6 7 7 7 7 7 4 4 ...
 $ Distance       : int  41571 37737 37694 37

In [37]:
%%R

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%


In [38]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [39]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1   NonRefAllele            1.000000          1.000000   0.145564
2       BaseQual            0.840335          0.840335   0.122322
3 NormalCoverage            0.481263          0.481263   0.070054
4      RefAllele            0.476538          0.476538   0.069367
5 TumourCoverage            0.417652          0.417652   0.060795

---
                     variable relative_importance scaled_importance percentage
54 GenomicLocation.intergenic            0.025433          0.025433   0.003702
55          Trinucleotide.CTG            0.024666          0.024666   0.003590
56          Trinucleotide.ACC            0.023967          0.023967   0.003489
57          Trinucleotide.ATC            0.020988          0.020988   0.003055
58                 StrandBias            0.010258          0.010258   0.001493
59  Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [40]:
%%R
write.table(var_importance, file="IS2_variable_importance.csv")

In [41]:
var_importance = pd.read_csv(open('IS2_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [42]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.145564
2,BaseQual,0.840335,0.840335,0.122322
3,NormalCoverage,0.481263,0.481263,0.070054
4,RefAllele,0.476538,0.476538,0.069367
5,TumourCoverage,0.417652,0.417652,0.060795


In [43]:
GenomicFeatures_IS2[:5]

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,13142,13143,41,8,1670,56,70,0,21.0,TCA,2.567376,0.599010,intergenic,41571,0.375000
1,1,16976,16977,82,24,3377,126,142,0,37.5,GCC,2.468531,0.589109,ncRNA_exonic,37737,0.658730
2,1,17019,17020,89,38,3943,151,172,0,51.0,GCC,2.427586,0.589109,ncRNA_exonic,37694,0.509934
3,1,17384,17385,103,39,4198,164,170,0,44.0,CCC,2.463087,0.603960,ncRNA_intronic,37329,0.396341
4,1,18933,18934,36,16,1558,66,59,0,31.0,GCG,2.263158,0.589109,ncRNA_intronic,35780,0.363636


In [44]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.84033519029617298,
 'Distance': 0.19990961253643,
 'GCcontent': 0.047415621578693397,
 'GenomicLocation.UTR3': 0.088150478899478898,
 'GenomicLocation.UTR5': 0.14704309403896301,
 'GenomicLocation.downstream': 0.129680350422859,
 'GenomicLocation.exonic': 0.26233139634132396,
 'GenomicLocation.exonic;splicing': 0.062393821775913197,
 'GenomicLocation.intergenic': 0.025432664901018101,
 'GenomicLocation.intronic': 0.035168416798114797,
 'GenomicLocation.missing(NA)': 0.15208736062049899,
 'GenomicLocation.ncRNA_UTR3': 0.076114907860755907,
 'GenomicLocation.ncRNA_UTR5': 0.093190222978591905,
 'GenomicLocation.ncRNA_exonic': 0.081660911440849304,
 'GenomicLocation.ncRNA_intronic': 0.049832645803690005,
 'GenomicLocation.splicing': 0.065657995641231495,
 'GenomicLocation.upstream': 0.16966858506202698,
 'GenomicLocation.upstream;downstream': 0.077157720923423795,
 'HomopolymerRate': 0.094356887042522403,
 'MapQual': 0.16998106241226199,
 'NonRefAllele': 1.0,
 'NormalCoverag

In [45]:
GenomicFeatures_IS2.columns

Index([u'CHROM', u'POS', u'END', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'Distance', u'StrandBias'], dtype='object')

In [46]:
GenomicFeatures_IS2.dtypes

CHROM               object
POS                  int64
END                  int64
RefAllele            int64
NonRefAllele         int64
BaseQual             int64
TumourCoverage       int64
NormalCoverage       int64
MapQual            float64
ReadPosition       float64
Trinucleotide       object
HomopolymerRate    float64
GCcontent          float64
GenomicLocation     object
Distance             int64
StrandBias         float64
dtype: object

In [47]:
GenomicFeatures_IS2.iloc[0,13]

'intergenic'

In [49]:
GenomicFeatures_IS2[:2]
#print GenomicFeatures_IS1.iloc[0,2]

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,13142,13143,41,8,1670,56,70,0,21.0,TCA,2.567376,0.599010,intergenic,41571,0.37500
1,1,16976,16977,82,24,3377,126,142,0,37.5,GCC,2.468531,0.589109,ncRNA_exonic,37737,0.65873


In [50]:
noOfSample = GenomicFeatures_IS2.shape[0]

IS2_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_IS2.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_IS2.iloc[i,3]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_IS2.iloc[i,4]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_IS2.iloc[i,5]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_IS2.iloc[i,6]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_IS2.iloc[i,7]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_IS2.iloc[i,8]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_IS2.iloc[i,9]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_IS2.iloc[i,10], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_IS2.iloc[i,10])]
    HomopolymerRate= GenomicFeatures_IS2.iloc[i,11]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_IS2.iloc[i,12]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_IS2.iloc[i,13], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_IS2.iloc[i,13])]
    Distance    = GenomicFeatures_IS2.iloc[i,14]*var_imp_dict['Distance']
    StrandBias     = GenomicFeatures_IS2.iloc[i,15]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' Distance=',Distance
    #print 'StrandBias=',StrandBias
    IS2_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+Distance+StrandBias

In [51]:
#Sorting dictionary
IS2_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in IS2_Dict.items()],reverse=True)]

In [53]:
IS2_Dict_Frame = pd.DataFrame(IS2_Dict1, columns=['POS', 'Value'])

In [54]:
IS2_Dict_Frame.to_csv("IS2_Dict_Frame.csv")

In [55]:
IS2_Dict_Frame[:10]

,POS,Value
0,26451660,419579.245298
1,26635480,382661.017284
2,42393802,309000.153427
3,42600077,284540.668777
4,15884875,281261.535970
5,22534834,275902.253205
6,46392259,263471.406525
7,92309631,253339.061118
8,15641359,233106.197318
9,27479018,214028.010276


In [61]:
GenomicFeatures_IS2_1 = pd.concat([IS2.iloc[:,0:16], IS2_Y_Val_1],axis=1)

In [62]:
rows = IS2_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_IS2_Top20 = GenomicFeatures_IS2_1.loc[GenomicFeatures_IS2_1['POS'].isin(rows)]

[ 26451660  26635480  42393802  42600077  15884875  22534834  46392259
  92309631  15641359  27479018   9032545  46386829  46387231 121483327
  27906683  15102130    569430  42599543  42390670   7119016]


In [64]:
GenomicFeatures_IS2_Top20[:10]

,CHROM,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
48,1,569430,569431,3726,6,125824,7936,7763,23,65,TCT,2.468966,0.465347,NaN,2191,0.360005,0
1906,1,121483327,121483328,4416,11,148650,6612,6874,37,47,TCA,2.652482,0.435644,NaN,2764,0.470508,0
5268,10,42390670,42390671,2532,1196,110720,15501,15362,0,16,ATG,2.190476,0.331683,NaN,6907,0.275273,0
5271,10,42393802,42393803,8016,33,272965,83248,83608,12,54,TCA,2.212329,0.341584,NaN,3775,0.368694,0
5349,10,42599543,42599544,3581,18,116837,15653,15717,9,36,TTC,2.105960,0.361386,NaN,4161,0.729892,0
5350,10,42600077,42600078,7962,59,261847,66879,66465,3,82,TTT,2.183673,0.366337,NaN,3627,0.404507,0
16546,16,46386829,46386830,5105,12,171419,8267,8293,23,55,CCA,2.059603,0.361386,NaN,41159,0.516753,0
16547,16,46387231,46387232,2720,2335,161659,7736,7524,9,39,ACA,2.273973,0.331683,NaN,40757,0.423345,0
16548,16,46392259,46392260,8015,10,268754,29750,29555,29,50,TTC,2.156463,0.376238,NaN,35729,0.330689,0
23116,2,92309631,92309632,7970,6,272901,21546,21298,29,44,GCA,2.647887,0.386139,intergenic,4758,0.423327,0


In [60]:
GenomicFeatures_IS2_Top20.to_csv('GenomicFeatures_IS2_Top20.csv')